In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from trl import setup_chat_format

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('hf_token.key', 'r') as f:
    hf_token = f.read()

### Meta LLaMA 3 8B Instruct

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map={"":0}
)

In [ ]:
# model, tokenizer = setup_chat_format(model, tokenizer)

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

In [ ]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
print(prompt)

In [ ]:
# inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [ ]:
inputs['input_ids']

In [ ]:
inputs

In [ ]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    tokenizer.convert_tokens_to_ids("<|end_of_text|>")
]

In [ ]:
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

In [ ]:
response = outputs[0][inputs['input_ids'].shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

### Meta LLaMA 3 8B

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 256

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map={"":0})

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.69s/it]


In [16]:
input_text = "Who is the current president of United States?"

In [17]:
inputs = tokenizer(input_text, return_tensors='pt', padding='max_length', truncation=True).to(model.device)

In [18]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

In [19]:
outputs = model.generate(
    **inputs,
    max_new_tokens=tokenizer.model_max_length,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    num_return_sequences=1
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [20]:
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [22]:
print(text)

Who is the current president of United States?
The current president of the United States is Joe Biden. He was elected as the 46th president of the United States in 2020 and took office on January 20, 2021. Biden is a Democrat and has served as the vice president under Barack Obama from 2009 to 2017.
Biden was born on November 20, 1942, in Scranton, Pennsylvania. He is the oldest of four children and grew up in a middle-class family. After graduating from the University of Delaware and Syracuse University College of Law, Biden began his career as a lawyer. He was elected to the New Castle County Council in 1970 and served until 1972. In 1972, he was elected to the United States Senate, where he served for 36 years.
Biden has been involved in many issues during his time in politics. He has been a strong advocate for civil rights, gun control, and health care reform. He has also been a vocal critic of President Donald Trump and has been involved in many legal battles with the Trump admin